In [ ]:
%pip install selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.edge.service import Service  
from bs4 import BeautifulSoup
import pandas as pd

# Path to your WebDriver (update this path)
webdriver_path = "D:\\power_bi\\edgedriver_win64\\msedgedriver.exe"

# URL of the page containing the table
url = "https://merolagani.com/LatestMarket.aspx"

# Set up Selenium WebDriver
options = webdriver.EdgeOptions()
options.add_argument("--headless")  # Run in headless mode (no UI)
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

# Use Service to specify the WebDriver path
service = Service(webdriver_path)
driver = webdriver.Edge(service=service, options=options)

try:
    # Open the webpage
    driver.get(url)

    # Wait for the table to load
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "table[data-live='live-trading']"))
    )

    # Get the updated page source after JavaScript execution
    html = driver.page_source

    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # Locate the table using the data-live attribute
    table = soup.find("table", {"data-live": "live-trading"})
    if not table:
        raise Exception("Table not found on the page.")

    # Extract headers from the <thead>
    headers = [th.get_text(strip=True) for th in table.find("thead").find_all("th")]

    # Extract rows from <tbody>
    rows = []
    tbody = table.find("tbody")
    if tbody:
        for tr in tbody.find_all("tr"):
            cells = [td.get_text(strip=True) for td in tr.find_all("td")]
            if cells:  # Avoid empty rows
                rows.append(cells)

    # Check if data was extracted
    if not rows:
        raise Exception("No data found in the table.")

    # Create a pandas DataFrame
    df = pd.DataFrame(rows, columns=headers)

    # Save data to CSV
    df.to_csv("latest_market_data.csv", index=False)
    print("Data successfully saved to latest_market_data.csv")

    # Print DataFrame
    print(df)

except Exception as e:
    print(f"Error: {e}")

finally:
    # Close the WebDriver
    driver.quit()


In [ ]:
# Save data to CSV
df.to_csv("latest_market_data.csv", index=False)

print("Data successfully saved to latest_market_data.csv")
